In [1]:
cd "/content/drive/My Drive/Colab Notebooks/ml-dw-matrix/matrix_three"

/content/drive/My Drive/Colab Notebooks/ml-dw-matrix/matrix_three


In [2]:
!ls

data  day01.ipynb  day02.ipynb	day03.ipynb  day04.ipynb  day05.ipynb  logs


In [3]:
!pip install hyperopt

In [24]:
import pandas as pd
import numpy as np

import os
import datetime
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score
from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

input_shape = X_train.shape[1:]
num_classes = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
    tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)

    model.fit(
        X_train,
        y_train,
        batch_size=params_fit.get('batch_size', 128),
        epochs=params_fit.get('epochs', 5),
        verbose=params_fit.get('verbose', 1),
        validation_data=params_fit.get('validation_data', (X_train, y_train)),
        callbacks= [tensorboard_callback]
    )
    return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
    y_test_norm = np.argmax(y_test, axis=1)

    y_pred_prob = model_trained.predict(X_test)
    y_pred = np.argmax(y_pred_prob, axis=1)

    return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_classes):
    return Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(0.3),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),        
        MaxPool2D(),
        Dropout(0.3),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),        
        MaxPool2D(),
        Dropout(0.3),        
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(0.3),
        Dense(1024, activation='relu'),
        Dropout(0.3),         
        Dense(num_classes, activation='softmax')        
    ])

In [47]:
model = get_cnn_v5(input_shape, num_classes)
model_trained = train_model(model, X_train, y_train)

predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 10s 37ms/step - loss: 2.5702 - accuracy: 0.2644 - val_loss: 1.1616 - val_accuracy: 0.6175
Epoch 2/5
272/272 [==============================] - 5s 20ms/step - loss: 0.7377 - accuracy: 0.7584 - val_loss: 0.1389 - val_accuracy: 0.9568
Epoch 3/5
272/272 [==============================] - 5s 20ms/step - loss: 0.2596 - accuracy: 0.9209 - val_loss: 0.0664 - val_accuracy: 0.9809
Epoch 4/5
272/272 [==============================] - 5s 20ms/step - loss: 0.1628 - accuracy: 0.9505 - val_loss: 0.0407 - val_accuracy: 0.9880
Epoch 5/5
272/272 [==============================] - 5s 20ms/step - loss: 0.1210 - accuracy: 0.9643 - val_loss: 0.0243 - val_accuracy: 0.9928


0.972108843537415

In [48]:
model_trained.evaluate(X_test, y_test)

138/138 [==============================] - 1s 4ms/step - loss: 0.1255 - accuracy: 0.9721


[0.12546859681606293, 0.9721088409423828]

In [0]:
def get_model(params):
    return Sequential([
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_one']),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),        
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_two']),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
        Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),        
        MaxPool2D(),
        Dropout(params['dropout_cnn_block_three']),
        Flatten(),
        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_one']),
        Dense(1024, activation='relu'),
        Dropout(params['dropout_dense_block_two']),
        Dense(num_classes, activation='softmax')        
    ])

In [0]:
def func_obj(params):
    model = get_model(params)
    model.compile(loss='categorical_crossentropy', optimizer='Adam', metrics=['accuracy'])

    model.fit(
        X_train,
        y_train,
        batch_size=int(params.get('batch_size', 128)),
        epochs=5,
        verbose=0
    )

    score = model.evaluate(X_test, y_test, verbose=0)
    accuracy = score[1]
    print(params, 'accuracy={}'.format(accuracy))

    return {
        'loss': -accuracy,
        'status': STATUS_OK,
        'model': model
        }

In [51]:
space = {
    'batch_size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.6),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch_size': 100.0, 'dropout_cnn_block_one': 0.4846302209546583, 'dropout_cnn_block_three': 0.3279828414138885, 'dropout_cnn_block_two': 0.35785226287054606, 'dropout_dense_block_one': 0.5798945069988872, 'dropout_dense_block_two': 0.42226723204688044}
accuracy=0.9546485543251038
{'batch_size': 160.0, 'dropout_cnn_block_one': 0.47992920390779775, 'dropout_cnn_block_three': 0.437668049605998, 'dropout_cnn_block_two': 0.3443479774885832, 'dropout_dense_block_one': 0.353806989220146, 'dropout_dense_block_two': 0.6008209835032068}
accuracy=0.9319728016853333
{'batch_size': 110.0, 'dropout_cnn_block_one': 0.45548280723549983, 'dropout_cnn_block_three': 0.4222374959103782, 'dropout_cnn_block_two': 0.4077913185531572, 'dropout_dense_block_one': 0.6194627790434108, 'dropout_dense_block_two': 0.5211176448931284}
accuracy=0.960090696811676
{'batch_size': 190.0, 'dropout_cnn_block_one': 0.44870661747332485, 'dropout_cnn_block_three': 0.33809851845476785, 'dropout_cnn_block_two': 0.3497959986648

In [0]:
!git status